<br />

<div style="text-align: center;">
<font size="7">TabNet</font>
<br /> 
<br /> 

    
</div>
<br />

<div style="text-align: right;">
<font size="4">2021/02/04</font>
<br />
<font size="4">Ryutaro Hashimoto</font>
</div>

___

# Summary
Run Sagemaker using a container with your own model.
Implement each file taking care of its format.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#AWS-Configure-settings" data-toc-modified-id="AWS-Configure-settings-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>AWS Configure settings</a></span></li><li><span><a href="#Install-Docker-for-Mac" data-toc-modified-id="Install-Docker-for-Mac-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Install Docker for Mac</a></span></li><li><span><a href="#Create-Container-Image" data-toc-modified-id="Create-Container-Image-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Create Container Image</a></span><ul class="toc-item"><li><span><a href="#train.py" data-toc-modified-id="train.py-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>train.py</a></span></li><li><span><a href="#predictor.py" data-toc-modified-id="predictor.py-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>predictor.py</a></span></li><li><span><a href="#model.py" data-toc-modified-id="model.py-1.3.3"><span class="toc-item-num">1.3.3&nbsp;&nbsp;</span>model.py</a></span></li></ul></li><li><span><a href="#Push-the-created-image-to-ECR." data-toc-modified-id="Push-the-created-image-to-ECR.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Push the created image to ECR.</a></span></li><li><span><a href="#wrapperfor-running-sagemaker" data-toc-modified-id="wrapperfor-running-sagemaker-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>wrapperfor running sagemaker</a></span></li></ul></li><li><span><a href="#Training" data-toc-modified-id="Training-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training</a></span><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setting</a></span><ul class="toc-item"><li><span><a href="#Setting-for-instance" data-toc-modified-id="Setting-for-instance-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Setting for instance</a></span></li><li><span><a href="#settting-for-tag" data-toc-modified-id="settting-for-tag-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>settting for tag</a></span></li><li><span><a href="#Specify-the-input-file" data-toc-modified-id="Specify-the-input-file-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Specify the input file</a></span></li><li><span><a href="#setting-for-metrics" data-toc-modified-id="setting-for-metrics-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>setting for metrics</a></span></li><li><span><a href="#setting-for-hyperparameters" data-toc-modified-id="setting-for-hyperparameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>setting for hyperparameters</a></span></li></ul></li><li><span><a href="#Execute" data-toc-modified-id="Execute-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Execute</a></span></li></ul></li><li><span><a href="#Deploy" data-toc-modified-id="Deploy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Deploy</a></span></li><li><span><a href="#Prediction" data-toc-modified-id="Prediction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Prediction</a></span><ul class="toc-item"><li><span><a href="#Prepare-sample-data" data-toc-modified-id="Prepare-sample-data-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prepare sample data</a></span></li><li><span><a href="#Prediction-in-CSV-format" data-toc-modified-id="Prediction-in-CSV-format-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Prediction in CSV format</a></span></li><li><span><a href="#Prediction-in-JSON-format" data-toc-modified-id="Prediction-in-JSON-format-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Prediction in JSON format</a></span></li></ul></li><li><span><a href="#Delete-the-endpoint" data-toc-modified-id="Delete-the-endpoint-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Delete the endpoint</a></span></li></ul></div>

## Preparation

### AWS Configure settings

Do this in advance. See __`0_AWS_access`__.

### Install Docker for Mac
When using Local Mode, make /opt/ accessible by docker.

### Create Container Image

Tree in the container

    .
    |-- Dockerfile
    |-- build_and_push.sh
    `-- decision_trees
        |-- nginx.conf
        |-- predictor.py
        |-- serve
        |-- train
        `-- wsgi.py- wsgi.py

* __`Dockerfile`__ Set up the environment in the container, install modules, mount directories, etc.
* __`build_and_push.sh`__ Push the image to ECR
* __`model`__ The directory where the model is stored.

Model Contents

* __`nginx.conf`__ the configuration file for the nginx front-end. 
* __`serve`__ Code launching the endpoint.
* __`wsgi.py`__ Wrapper when launching the endpoint.　
* __`train`__ Import the model from __model.py__, run the training, and save it. Needs to be edited.
* __`predictor.py`__ Executed when the endpoint is started and called.Needs to be edited.
* __`model.py`__ the details of the model, preprocessing, and other behaviors are described in a modular manner.　Needs to be edited, mainly.

#### train.py

Change the model to import. The rest is basically the same

The behavior is to execute the train function, save the model to the specified path after training, and return 0.

#### predictor.py

Change the model to import. The rest is basically the same

The behavior is as follows: After retrieving data with flask.Request, ScoringService.get_model retrieves the stored model. Then, after predicting with ScoringService.predict, the prediction result is sent with flask.

#### model.py

The class for the model should be roughly as follows.
```
class Model:
    def __init__(self, params=None):
        Get hyperparameters

    def fit(self, X):
        Load and train data frames.
        
    def predict(self, X):
        Read and predict data frames.

    def save_model(self, file_path):
        Save the model

    def load_model(self, file_path):
        Load the model.
```

### Push the created image to ECR.

In the /container, run


```
./build_and_push.sh <image_name>
```

in a terminal, then automatically upload the file to ECR.

### wrapperfor running sagemaker
Wrapping sgemaker so that it automatically starts and runs like schikit-learn(__`wrapper_container.py`__)

In [1]:
import sys
sys.path.append('../')
from util.wrapper_container import RunSageMaker

In [2]:
import inspect
inspect.getsourcelines(RunSageMaker) 

(['class RunSageMaker:\n',
  '    """\n',
  '    インスタンスを立ち上げ学習を実行する。\n',
  '    学習結果を登録しているmetricsを含んだpandas DataFrameで返す。\n',
  '    """\n',
  '\n',
  '    def __init__(self,\n',
  '                 image_uri,\n',
  '                 role,\n',
  '                 params,\n',
  '                 metric_definitions,\n',
  '                 tag_config,\n',
  '                 instance_config=None):\n',
  '        self.estimator = None\n',
  '        self.image_uri = image_uri\n',
  '        self.role = role\n',
  '        self.metric_definitions = metric_definitions\n',
  '        self.tag_config = tag_config\n',
  '\n',
  '        if params is None:\n',
  '            self.params = {}\n',
  '        else:\n',
  '            self.params = params\n',
  '        if instance_config is None:\n',
  '            self.instance_config = {}\n',
  '        else:\n',
  '            self.instance_config = instance_config\n',
  '\n',
  '    def fit(self, input, output_path, job_name=None, tag_config=

## Training

In [3]:
import json
import pandas as pd
import sagemaker



### Setting

#### Setting for instance

In [4]:
instance_config = {
    'framework_version': '1.4.0',
    'py_version': 'py3',
    'instance_count': 1,
    'instance_type': 'local',
}

#### settting for tag

In [5]:
tag_config=[
            {"Key": "module", "Value": "Pytorch"},
            {"Key": "model", "Value": "TabNet"},
            {"Key": "container", "Value": "original"}
            ]

#### Specify the input file

In [6]:
input = {
    'base_dir': 's3 path',
    'train_data': '/train.csv',
    'valid_data': '/valid.csv'
}
output_path = "s3 path"

#### setting for metrics

In [7]:
metric_definitions=[
                    {'Name': 'rmse', 
                    'Regex': 'rmse: ([-+]?[0-9\\.]+)'},
                    {'Name': 'val_mse:', 
                    'Regex': 'val_0_mse: ([-+]?[0-9\\.]+)'}, 
                    {'Name': 'epoch:', 
                    'Regex': 'epoch ([-+]?[0-9\\.]+)'}, 
                    ]

#### setting for hyperparameters

In [8]:
params_pretrain = json.dumps(
                    {
                    'batch_size': 128,
                    'virtual_batch_size': 128,
                    'patience': 30,
                    'max_epochs': 100,
                    'optimizer_params': {'lr': 0.01, 'weight_decay': 1e-05},
                    'n_steps': 1,
                    }
                )

params = json.dumps(
                    {
                     'n_a':8,
                     'n_d': 8,
                     'scheduler_params': {'max_lr': 0.01, 'steps_per_epoch': 2, 'epochs': 500},
                     'max_epochs':  50,
                     'batch_size': 64,
                     'virtual_batch_size': 64,
                     'patience': 100,
                    }
                )

categorical_columns = ['Open', 'Promo', 'SchoolHoliday', 'month', 'date']

In [9]:
params = {
    'params-pretrain': params_pretrain, 
    'params': params, 
    'target': 'Sales', 
    'categorical-columns': ' '.join(map(str, categorical_columns)), 
}

### Execute

In [10]:
estimator = RunSageMaker(
              image_uri='005242542034.dkr.ecr.us-west-2.amazonaws.com/tabnet-regressor',
              role='Role',
              params = params,
              metric_definitions=metric_definitions,
              tag_config=tag_config,
              instance_config=instance_config
             )

In [11]:
import datetime
jobname = 'tabnet-sagemaker-' + str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
estimator.fit(input=input, output_path=output_path, job_name=jobname)

Creating tmpg3b5y5t__algo-1-71k4e_1 ... 
Attaching to tmpg3b5y5t__algo-1-71k4e_12mdone
algo-1-71k4e_1  | Starting the training.
algo-1-71k4e_1  | {'params-pretrain': '{"batch_size": 128, "virtual_batch_size": 128, "patience": 30, "max_epochs": 100, "optimizer_params": {"lr": 0.01, "weight_decay": 1e-05}, "n_steps": 1}', 'params': '{"n_a": 8, "n_d": 8, "scheduler_params": {"max_lr": 0.01, "steps_per_epoch": 2, "epochs": 500}, "max_epochs": 50, "batch_size": 64, "virtual_batch_size": 64, "patience": 100}', 'target': 'Sales', 'categorical-columns': 'Open Promo SchoolHoliday month date', 'input': '{"base_dir": "s3://poctesthashimoto/data/rossmann/", "train_data": "/train.csv", "valid_data": "/valid.csv"}'}
algo-1-71k4e_1  | /usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internall

## Deploy

In [12]:
# estimator.deploy(instance_type='ml.m4.xlarge')
estimator.deploy(instance_type='local')

Attaching to tmp1t9w5g74_algo-1-f7lmx_1
algo-1-f7lmx_1  | Starting the inference server with 4 workers.
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [10] [INFO] Starting gunicorn 20.0.4
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [10] [INFO] Using worker: gevent
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [13] [INFO] Booting worker with pid: 13
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [14] [INFO] Booting worker with pid: 14
algo-1-f7lmx_1  | [2021-02-08 00:52:13 +0000] [15] [INFO] Booting worker with pid: 15
algo-1-f7lmx_1  | [2021-02-08 00:52:14 +0000] [16] [INFO] Booting worker with pid: 16
algo-1-f7lmx_1  | /usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally

## Prediction

Both CSV and JSON are supported.
CSV is input in pandas.DataFrame as in table_data_processing, and a list of predicted values is output.

### Prepare sample data

In [13]:
# read csv
test=pd.read_csv("data/test.csv", index_col=0)
# drop the label column in the training set
test.drop(test.columns[[1]],axis=1,inplace=True)
test.head()

,DayOfWeek,Customers,Open,Promo,SchoolHoliday,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,...,StoreType_nan,Assortment_b,Assortment_c,Assortment_nan,"PromoInterval_Jan,Apr,Jul,Oct","PromoInterval_Mar,Jun,Sept,Dec",PromoInterval_nan,year,month,date
156,3,653,1,0,0,570,11,8,1,13,...,0,0,0,0,1,0,0,2015,2,24
155,4,509,1,0,0,570,11,8,1,13,...,0,0,0,0,1,0,0,2015,2,25
154,5,517,1,0,0,570,11,8,1,13,...,0,0,0,0,1,0,0,2015,2,26
153,6,378,1,0,0,570,11,8,1,13,...,0,0,0,0,1,0,0,2015,2,27
152,7,0,0,0,0,570,11,8,1,13,...,0,0,0,0,1,0,0,2015,3,0


### Prediction in CSV format

In [14]:
res = estimator.predict(test)

algo-1-f7lmx_1  | Invoked with 157 records
algo-1-f7lmx_1  | 172.28.0.1 - - [08/Feb/2021:00:52:17 +0000] "POST /invocations HTTP/1.1" 200 1503 "-" "-"


In [15]:
res

[8.406941,
 8.628193,
 8.58601,
 8.628108,
 4.3018703,
 7.833139,
 8.228649,
 7.9546666,
 8.068433,
 7.979149,
 7.376253,
 5.463338,
 2.511085,
 5.541976,
 7.405779,
 7.9194565,
 7.9253063,
 8.054446,
 6.9166365,
 7.111681,
 8.730817,
 8.564101,
 8.625599,
 8.620312,
 8.459248,
 8.263453,
 2.9758162,
 5.640297,
 8.083109,
 8.48851,
 8.628193,
 8.628193,
 8.628193,
 2.5356114,
 5.766569,
 6.163084,
 6.930984,
 0.5181865,
 5.99502,
 4.276666,
 0.077486955,
 0.084979534,
 2.3834672,
 3.2849617,
 4.803648,
 7.7641993,
 5.7661,
 4.8382845,
 8.523142,
 8.318998,
 8.59733,
 8.530217,
 8.316623,
 7.5074058,
 1.520764,
 3.9589303,
 7.729063,
 8.297945,
 8.583028,
 8.448399,
 8.606077,
 3.4732022,
 7.849145,
 8.628193,
 8.628193,
 8.400873,
 5.8168726,
 3.241353,
 4.3632483,
 7.624977,
 7.851515,
 8.064615,
 7.1305585,
 7.181634,
 4.662574,
 0.19406417,
 2.4814773,
 6.620899,
 4.2077594,
 4.1782303,
 8.062523,
 6.5782123,
 3.8799593,
 5.6154814,
 8.519918,
 8.628107,
 8.628193,
 8.59973,
 8.3134

In [16]:
estimator.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


### Prediction in JSON format

In [17]:
estimator.deploy(instance_type='ml.p2.xlarge', 
                 serializers=sagemaker.serializers.JSONSerializer(), 
                 deserializer=sagemaker.deserializers.JSONDeserializer())

Attaching to tmpwzp6nbew_algo-1-iswbj_1
algo-1-iswbj_1  | Starting the inference server with 4 workers.
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [10] [INFO] Starting gunicorn 20.0.4
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [10] [INFO] Listening at: unix:/tmp/gunicorn.sock (10)
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [10] [INFO] Using worker: gevent
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [13] [INFO] Booting worker with pid: 13
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [14] [INFO] Booting worker with pid: 14
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [15] [INFO] Booting worker with pid: 15
algo-1-iswbj_1  | [2021-02-08 00:52:22 +0000] [16] [INFO] Booting worker with pid: 16
algo-1-iswbj_1  | /usr/local/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally

In [18]:
res = estimator.predict(test.to_json(orient='split'))

algo-1-iswbj_1  | 172.28.0.1 - - [08/Feb/2021:00:52:25 +0000] "GET /ping HTTP/1.1" 200 1 "-" "-"
algo-1-iswbj_1  | Invoked with 157 records
algo-1-iswbj_1  | 172.28.0.1 - - [08/Feb/2021:00:52:25 +0000] "POST /invocations HTTP/1.1" 200 3351 "-" "-"


In [19]:
res

{'predictions': [[8.406941413879395],
  [8.628192901611328],
  [8.586009979248047],
  [8.628108024597168],
  [4.301870346069336],
  [7.833138942718506],
  [8.228649139404297],
  [7.954666614532471],
  [8.068432807922363],
  [7.979148864746094],
  [7.376253128051758],
  [5.4633378982543945],
  [2.511085033416748],
  [5.541975975036621],
  [7.405778884887695],
  [7.919456481933594],
  [7.92530632019043],
  [8.05444622039795],
  [6.9166364669799805],
  [7.11168098449707],
  [8.730816841125488],
  [8.564101219177246],
  [8.625598907470703],
  [8.620311737060547],
  [8.459247589111328],
  [8.263452529907227],
  [2.975816249847412],
  [5.640296936035156],
  [8.083108901977539],
  [8.488510131835938],
  [8.628192901611328],
  [8.628192901611328],
  [8.628192901611328],
  [2.535611391067505],
  [5.766569137573242],
  [6.163084030151367],
  [6.930984020233154],
  [0.5181865096092224],
  [5.995019912719727],
  [4.276666164398193],
  [0.07748695462942123],
  [0.08497953414916992],
  [2.3834671974

## Delete the endpoint
Remember to delete them.

In [21]:
estimator.delete_endpoint()

Gracefully stopping... (press Ctrl+C again to force)


In [22]:
# End of File